# Walmart Agent - Databricks Deployment

Este notebook prepara e registra o Agente Walmart no MLflow para deploy no Databricks Model Serving.

## Pré-requisitos
1. Cluster rodando (mlflow, langchain instalados).
2. `databricks-sql-connector` instalado.
3. Variáveis de ambiente configuradas (OPENAI_API_KEY, DATABRICKS_*).

In [0]:
# langchain-google-vertexai
%pip install langchain==0.3.25  langgraph databricks-sql-connector pandas python-dotenv langchain-databricks==0.1.0 langchain-openai mlflow

In [0]:
dbutils.library.restartPython()

In [0]:
%pip install dotenv

In [0]:
import os
import sys
from dotenv import load_dotenv

# Load env vars from .env if present (Local dev)
load_dotenv()

# In Databricks, we often use Secrets, but for this notebook execution, we ensure variables are set.
# os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="my-scope", key="openai-key")

print("Environment checked.")

## Definição do Wrapper MLflow

Criamos uma classe `WalmartAgentWrapper` que encapsula o agente. Esta classe é serializada pelo MLflow e usada pelo Model Serving.

In [0]:
import mlflow
import pandas as pd
from mlflow.models import infer_signature

class WalmartAgentWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Executado uma vez quando o container do modelo inicia.
        """
        print("Loading Walmart Agent Context...")
        
        # Importações dentro do contexto para garantir que as dependências estejam carregadas
        from examples.walmart_agent.agent import WalmartAgent
        import logging
        
        logging.basicConfig(level=logging.INFO)
        
        # Inicializa o Agente
        # As credenciais devem vir de variáveis de ambiente configuradas no Endpoint de Serving
        try:
            self.agent = WalmartAgent()
            print("Walmart Agent Initialized Successfully.")
        except Exception as e:
            print(f"Error initializing agent: {e}")
            raise e

    def predict(self, context, model_input):
        """
        Executado a cada requisição.
        Esperamos um DataFrame pandas com uma coluna 'messages' ou 'input'.
        """
        # Normalização da entrada
        if isinstance(model_input, pd.DataFrame):
            if "messages" in model_input.columns:
                user_message = model_input["messages"].iloc[0]
            elif "input" in model_input.columns:
                user_message = model_input["input"].iloc[0]
            else:
                user_message = str(model_input.iloc[0, 0])
        elif isinstance(model_input, dict):
            user_message = model_input.get("messages", model_input.get("input", str(model_input)))
        else:
            user_message = str(model_input)
            
        # Executa o chat
        try:
            response = self.agent.chat(user_message)
            return [response]
        except Exception as e:
            return [f"Error processing request: {str(e)}"]

## Teste Local do Wrapper
Validamos se o wrapper funciona antes de registrar.

In [0]:
# Teste rápido (opcional, requer env vars configuradas neste notebook)
try:
    wrapper = WalmartAgentWrapper()
    wrapper.load_context(None)
    
    test_input = pd.DataFrame({"messages": ["How are the sales in North region?"]})
    response = wrapper.predict(None, test_input)
    print("Test Response:", response)
except Exception as e:
    print("Skipping local test due to error (check env vars):", e)

## Registrar no MLflow

Empacotamos o código (`src` e `examples`) junto com o modelo.

In [0]:
# Definição do Ambiente Conda/Pip
conda_env = {
    "channels": ["defaults"],
    "dependencies": [
        "python=3.11.10",
        "pip",
        {
            "pip": [
                "langchain",
                "langgraph",
                "pandas<2.2.0",
                "requests",
                "python-dotenv",
                "pydantic",
                "pillow",
                "databricks-sql-connector",
                "langchain-community",
                "databricks-langchain",
                "langchain-openai",
                "mlflow",
                "google-cloud-aiplatform",
                "langchain-google-vertexai"


                # "mlflow",
                # "langchain==0.3.25",
                # "langgraph==0.4.8",
                # "langchain-openai",
                # "langchain-databricks==0.1.0",
                # "databricks-sql-connector==3.1.0",
                # "pandas<2.2.0",
                # "python-dotenv==1.1.0",
                # "langgraph-checkpoint==3.0.1",
                # "langgraph-prebuilt==1.0.5",
                # "cloudpickle==3.0.0"
                
                # "langchain-google-vertexai==2.0.25"
            ]
        }
    ]
}

# Exemplo de Entrada/Saída para Assinatura
input_example = pd.DataFrame({"messages": ["Show me sales data"]})
signature = infer_signature(input_example, ["The sales data is..."])

# Ajuste o caminho do experimento se necessário
# mlflow.set_experiment("/Users/seu_usuario/walmart_agent")

with mlflow.start_run(run_name="walmart_agent_v1") as run:
    mlflow.pyfunc.log_model(
        artifact_path="walmart_agent_model",
        python_model=WalmartAgentWrapper(),
        conda_env=conda_env,
        signature=signature,
        input_example=input_example,
        # Inclui o código fonte para que o Wrapper consiga importar 'examples' e 'src'
        code_path=["examples", "src"]
    )
    
    print(f"Model logged successfully!")
    print(f"Run ID: {run.info.run_id}")
    print(f"Model URI: runs:/{run.info.run_id}/walmart_agent_model")

## Próximos Passos (Deploy)

1. Vá para a aba **Experiments** ou **Models** no Databricks.
2. Encontre o modelo registrado acima.
3. Clique em **Deploy to Model Serving**.
4. Em **Advanced Configuration** -> **Environment Variables**, adicione:
   - `OPENAI_API_KEY`: `{{secrets/seu_scope/openai_key}}`
   - `DATABRICKS_SERVER_HOSTNAME`: `...`
   - `DATABRICKS_HTTP_PATH`: `...`
   - `DATABRICKS_TOKEN`: `...`
   - `DEPLOY_TYPE`: `databricks`
5. Aguarde o Endpoint ficar **Ready**.
6. Teste via aba **Serving** ou via REST API.